Import Stuff

In [1]:
import ssl
import re

# from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired
# from hdbscan import HDBSCAN
# from sentence_transformers import SentenceTransformer
# from umap import UMAP

# from sklearn.datasets import fetch_20newsgroups
# from sklearn.cluster import KMeans

import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob
from textblob_de import TextBlobDE
# from datasets import load_dataset

# import pickle
import importlib
import functions
import requests





This cell contains the necessary imports and setups required for the project, including libraries for data manipulation, visualization, natural language processing, and API interaction. It also defines the file path and loads the DeepL API key for translation purposes.

In [13]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


This cell initializes necessary variables, downloads required NLTK resources, and loads the DeepL API key for translation. It also sets up the API endpoint URL for DeepL and defines a list of political parties for further processing.

Edit Text documents

In [25]:
df_manifesto_raw = pd.read_csv("data/parties/All_Manifestos.csv")
df_manifesto_tokenized = pd.read_csv("data/parties/All_Manifestos_tokenized.csv")
cmp_categories = pd.read_csv(file_path  + "cmp_categories.csv")
# df_manifesto["year"] = pd.to_numeric(df_manifesto["year"], errors='coerce')

This cell loads the manifesto data and related categories from CSV files into pandas DataFrames. It prepares the data for further processing by reading the manifesto text, tokenized text, and category mappings. The `file_path` variable is used to construct the path for the category file.

In [26]:
df_cdu = df_manifesto_raw[df_manifesto_raw['party'] == 'CDU']

In [28]:
# Apply the function to df_manifesto
df_manifesto = process_manifesto_data(df_manifesto_raw)
# df_manifesto_tokenized = process_manifesto_data(df_manifesto_tokenized)

In [46]:
def drop_entries_by_conditions(df):
    """
    Drops rows from the dataframe where year > 1997 and cmp_code is NaN.

    Parameters:
        df (pd.DataFrame): The input dataframe.

    Returns:
        pd.DataFrame: The filtered dataframe.
    """
    return df[ ~ ((df['cmp_code'].isna()) & (df['year'] > 1997))]

In [29]:
# Merge df_manifesto with cmp_categories to add the description_md column
df_manifesto = df_manifesto.merge(cmp_categories[['code', 'label', 'description_md']], 
                                  left_on='cmp_code', 
                                  right_on='code', 
                                  how='left')

df_manifesto.rename(columns={"label": "category"}, inplace=True)

# Drop the redundant 'code' column after the merge
df_manifesto.drop(columns=['code'], inplace=True)

In [31]:
# Apply the function to df_manifesto to reduce noise
# df_manifesto = drop_na_after_date(df_manifesto, 1997)
df_manifesto = df_manifesto[ ~ ((df_manifesto['cmp_code'].isna()) & (df_manifesto['year'] > 1997))]

In [ ]:
df_manifesto_final["sentiment"] = df_manifesto_final["text"].apply(lambda x: TextBlobDE(x).sentiment.polarity)

In [ ]:
test = TextBlobDE(df_manifesto_final["text"][2000])
test.sentiment.polarity


In [32]:
df_manifesto.to_pickle('data\df_manifesto.pkl')